# Colorization using pix2pix GAN

**term project for course pa228 ML for image processing at FI MUNI**

**Marek Kadlčík**

**učo 485294**

## Overview

The project is an implementation of a [pix2pix GAN](https://arxiv.org/abs/1611.07004) and the goal is to colorize black and white images. The code is based on Keras and Tensorflow2 and trains the GAN on a subset of [imagenet dataset](https://www.image-net.org/) downloaded from [kaggle object localization challenge](https://www.kaggle.com/competitions/imagenet-object-localization-challenge/data) containing of approximately 1.2 million images.

Pix2pix is conditional GAN. The architecture consists of a U-net generator and a fully convolutional discriminator. In our case, the generator takes a grayscale image as an input and produces a colorized version. The discriminator takes a colorful image and scores its believability.

There are few specific design choices. First, the generator is trained not only to fool the discriminator, but also with an l1 loss computed directly against the original colorful image. The authors of pix2pix paper argue it produces better results, but they do not focus on colorization problem specically, so this might be worth investigating further.

Second design choice is that the discriminator does ont predict a single score, but a small 2d grid of scores, each corresponding to one patch in the input image.

Last notable choice is that the generator does not take a noise vector as an input. All randomness in the generated colorization is caused by dropout in generator, which authors apply during both training and inference. Authors state that they experimented with noise vector and argue that the NN learned to ignore it anyways so why bother.

## Implementation

My implementation is based on keras and tensorflow and losely follows the pix2pix paper. Some implementation details are based on [Tensorflow tutorial](https://www.tensorflow.org/tutorials/generative/pix2pix). Because the pix2pix paper is relatively old in the GAN field, I also searched the internet about best practises and tricks for training GANs and read these sources: [ganhacks repo](https://github.com/soumith/ganhacks), [ML master blog](https://machinelearningmastery.com/how-to-train-stable-generative-adversarial-networks/), [this reddit discussion](https://www.reddit.com/r/MachineLearning/comments/i085a8/d_best_gan_tricks/) and [Jonathan Hui's blog post](https://towardsdatascience.com/gan-ways-to-improve-gan-performance-acf37f9f59b). The things I've done differently than the pix2pix paper are:

1) Label smoothing
1) Larger batch size
1) Larger input size
1) Color space
1) Hyperparameter selection
1) Training data


### Label smoothing
Label smoothing is not mentioned in the pix2pix paper, so I suppose they didn't use it. With label smoothing, the discriminator is trained to predict scores for example 0.9 for real images and 0.1 for fakes instead of 1 and 0. This is used because if the discriminator becomes too convinced about the score, then no changes on the input would change his prediction. This would cause very low gradients when backpropagating from discriminator to generator. For our purposes it suffices to smooth the labels for fake images and not reals (one-sided label smoothing), but standard label smoothing is slightly easier to implement (only a parameter of binary cross entropy), so I went with that.


### Larger batch size

The general consensus is that large batch size is benefitial for the stability and convergence of GAN training. In pix2pix, they only used batch size 4 (possibly a HW contraint, pix2pix paper was originally published in 2016 , but I could train on GPU's with large memory so I experimented with 64, 128 and 256 batch size and ended up using 64.


### Larger input size

Pix2pix uses 256x256 resolution, I used 512x512 just because I had enough GPU memory to do that. However, a large portion of Imagenet dataset is smaller than that so it was partly wasteful.


### Color space

In pix2pix, RGB representation of images is used. My consultant, RNDr. Filip Lux, warned me that it might not be the most suitable representation for colorization task, especially when l1 loss is used for generator in addition to GAN loss. Many publications focused on colorization use CIELAB color space instead.

Things like cars, clothes or buildings do not have just one possible colorization. If generator uses blue to colorize car that was red in the original photo, l1 will strongly penalize it and this results in overly grey and undersaturated outputs where colorization is ambiguous. In CIELAB representation, the problem is not as prominent.
I also experimented with HSV color space but it turned out just bad.


### Hyperparameter selection

I had a hard time making the GAN produce results that would at least convince me that the implementation is correct. I experimented with different combination of discriminator and generator learning rates and the relative weight of the l1 loss compared to GAN loss.

I started with hyperparameters from the pix2pix paper, but after a series of unsuccessful attempts and experimenting I started to look online for some hints on tuning the hyperparameters. I have found a [colorization thesis by Lucie Novotná](https://is.muni.cz/th/cawr6/) that also describes struggling with hyperparameters and shows what combination worked well. I tried them and also got promising results. The main difference from the pix2pix paper is that pix2pix uses the same learning rate for discriminator and generator (although generator loss is "boosted" by hundredfold l1 loss), while Lucie found that generator should have around 20 times higher learning rate than discriminator.


### Training data

The GAN in pix2pix was trained for 6 epochs on the the Imagenet that contained around 1.2 million images at that time. Nowadays, Imagenet is much larger, but I used a subset of approximately same size as was the original imagenet. However, I used batch size 64, so one my epoch has 16x less updates than their epoch.

## Outputs

### Baseline

My baseline model has the same architecture as the generator in the GAN model. However, it uses RGB representation and is trained using L1 loss.

See outputs on [weights and biases](https://wandb.ai/markcheeky/colorgan/runs/1wbhzlfo?workspace=user-markcheeky). A lot of pictures have green grass or blue sky, but most objects are jut grey.

### GAN

see outputs on [weights and biases](https://wandb.ai/markcheeky/colorgan/runs/1p530utx?workspace=user-markcheeky). Some pictures look ok, but most have some artifacts or can be easily distuinguished from the real photos. Not very satisfactory. I'm quite sure that the implementation is correct, but a lot more work in hyperparameter selection needs to be done to get better result.

## Lessons learned


### Implementing a GAN is easy, making it work is not

In my experience, you can set bad hyperparameters like learning rate for training a supervised task and the NN will probably learn okish with SGD and fine with smarter optimizer such as Adam. GANs will not forgive things that standard NN would. They can just wander around, show no signs that something is wrong and still not converge. Or maybe one must have some experience to detect those signs.


### Log everything

After some time I spent debugging having no clue what is wrong, I decided to log everything. Size of every loss, histogram of model parameters, norm of gradients, average discriminator score on fakes and real images, discriminator accuracy... Still, I didn't log the gradient norm of each generator loss individually and I think it would come handy. Next time I train GAN, I will log **everything** from the start.


### Not everything is backprop-friendly

I found a tensorflow package that could convert RGB image tensor to CIELAB color space (and back) and wanted to use it only for computing the L1 loss. While the function worked perfectly fine and I checked many times that the forward pass is functioning just as expected, the training always collapsed after first batch because the packpropagation produced nan's. I then needed to rewrite the NN to work entirely in CIELAB space, and the takeaway is that not every differentiable tensor function is ready for backpropagation. 


### Colorization does not generalize very well

I didn't think about it ahead very much, but I didn't realize that the model cannot learn to colorize objects that are not represented enough in the training set, simply because the model does not know what color it should have. If it was trained oon everyday objects, but hasn't seen a tennis ball, it does not know that it is yellow. I think this could be solved however with additional "conditioning" of the input. Basically we would extract regions with similar color in the training time, aggregate it somehow and give it in addition to black-and-white photo to the generator. In inference time, human would pick several places on the image and select the color that should be (approximately) used in that region. I think that this approach could handle unseen object, give us some control but would still require little human input.


### Start small and specific

If I started again, I would use a small but specific dataset first, maybe just dog photos, or just shoes. I had a very slow feedback loop because I always run the training on imagenet, hoping that I would recognize whether it is learning or not. As I learned, it is very difficult to tell whether a GAN is on the right track or not, so experimenting on small non-diverse dataset is a must. 


### Streaming a dataset during training is... complicated

I first thought I would train on a HW where I had very limited storage. So, I started with a code that would download images on demand from a text file of url addreses during training. Turned out that it is quite difficult to get right. One need to make sure that the GPU is utilized and is not waiting all the time, design appropriate async/parallel downloading, handle all edge cases and errors that just would not occur with local dataset. And even if everything works, things could go wrong just because some network device / mechanism would recognize downloading so many images one by one at that frequency as a form of spamming or attack (especiall in cases like Open Images 2, where all urls refer to different subdomains of a single server). When I got access on HW with storage and downloaded the whole dataset locally, setting up a TF dataset was super easy (even though the prepared functions are not very flexible).